In [123]:
import os, errno
import re
import shutil
import tarfile
import nglview as nv
import MDAnalysis as mda
from MDAnalysis.analysis import align
from modeller_script.evaluate_modeller import evaluate_modeller

In [ ]:
def create_directory(directory):
    try:
        os.makedirs(directory)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    
    return directory

In [ ]:
def move_files(source, dest, file_name):
    files = os.listdir(source)
    for f in files:
        if re.match(file_name, f):
            shutil.move(source+f, dest)

### 1. prepare directory

In [2]:
PDB_CODE = '6b73'
SEQ = 'B'

In [3]:
folder_name = PDB_CODE+SEQ
path = '../'+folder_name

In [ ]:
directory = create_directory(path)

### 2. download pdb file and fasta sequence:

In [ ]:
file_name = PDB_CODE+'.pdb'
! (cd $directory && curl -O https://files.rcsb.org/download/$file_name)

In [ ]:
fasta_path = directory+'/'+PDB_CODE+'.fasta'
fasta_url = '"'+'https://www.rcsb.org/pdb/download/downloadFile.do?fileFormat=fastachain&compression=NO&structureId='+PDB_CODE+'&chainId='+SEQ+'"'        
! curl -o $fasta_path $fasta_url

### 3. create sequence:

In [ ]:
! (cd $directory && mod9.21 ../code/modeller_script/get_seq.py $PDB_CODE $SEQ)

### 4. create alignement:

In [ ]:
sequence = ''
sequence_path = directory+'/'+PDB_CODE+'.seq'
ali_path = directory+'/alignment.ali'

with open(sequence_path) as sequence_file:
    alignment_file = open(ali_path, 'w+')
    for i, line in enumerate(sequence_file):
        alignment_file.write(line)
        if i > 2:
            sequence += line
    sequence = sequence[:-2]    
    alignment_file.write('>P1;'+PDB_CODE+'_fill\n')
    alignment_file.write('sequence:::::::::')
    alignment_file.close()

### 5. compose template and target sequence:
	template: >P1;6b73
				structureX:6b73:
				- - - for missing residues
	target: >P1;6b73_fill
				sequence:::::::::
				insert residues where it missing;
				- - - for residues what are not needed

In [ ]:
! open -e $ali_path
! /Applications/Chimera.app/Contents/MacOS/chimera $directory/$file_name

### 6. run modeller:
 * knowns = sys.argv[1]
 * sequence = sys.argv[2]
 * num_models = sys.argv[3]
 * num_loops = sys.argv[4]

In [ ]:
pdb_fill = PDB_CODE+'_fill'
! (cd $directory && mod9.21 ../code/modeller_script/loopmodel.py $PDB_CODE $pdb_fill 20 4)

### 7. evaluate modeller result:

In [5]:
file_name = pdb_fill+'.'
best_model, best_loop = evaluate_modeller(file_name, loop=True)

            DOPE  ga341       molpdf                     name  norm_DOPE
5  -38197.539062    1.0   919.880127  6b73_fill.B99990006.pdb   -0.55493
7  -38087.347656    1.0   918.481445  6b73_fill.B99990008.pdb   -0.52706
3  -38080.039062    1.0   906.115295  6b73_fill.B99990004.pdb   -0.52521
14 -38067.464844    1.0   955.168335  6b73_fill.B99990015.pdb   -0.52203
13 -37928.148438    1.0   962.934448  6b73_fill.B99990014.pdb   -0.48679
9  -37927.113281    1.0   984.575317  6b73_fill.B99990010.pdb   -0.48653
18 -37926.093750    1.0   978.404114  6b73_fill.B99990019.pdb   -0.48627
16 -37916.238281    1.0   973.556519  6b73_fill.B99990017.pdb   -0.48378
17 -37885.402344    1.0   884.425964  6b73_fill.B99990018.pdb   -0.47598
2  -37876.269531    1.0   979.899719  6b73_fill.B99990003.pdb   -0.47367
12 -37875.011719    1.0  1070.296997  6b73_fill.B99990013.pdb   -0.47335
4  -37871.675781    1.0  1028.565430  6b73_fill.B99990005.pdb   -0.47251
11 -37859.902344    1.0  1005.030090  6b73_fill.B99

In [6]:
! /Applications/Chimera.app/Contents/MacOS/chimera $directory/$best_model $directory/$best_loop

In [19]:
def copy_best(source, dest, file_name, best):
    files = os.listdir(source)
    for f in files:
        if re.match(best, f):
            shutil.copy(source+'/'+f, dest+file_name+'pdb')

In [20]:
model_folder = create_directory(directory+'/models')
move_files(directory+'/', model_folder, file_name, best_model)
copy_best(model_folder, directory+'/', file_name, best_model)
copy_best(model_folder, directory+'/', file_name[:-1]+'_loop.', best_loop)

In [151]:
! cp ./modeller_script/loopmodel.log $model_folder

### 8. upload to OPM:
PDB_CODE_fill.pdb
https://opm.phar.umich.edu/ppm_server

In [21]:
new_opm_name = directory+'/'+PDB_CODE+'_fill_opm.pdb'
! cp /Users/evabertalan/Downloads/*_fill.pdb $new_opm_name

In [24]:
new_loop_opm_name = directory+'/'+PDB_CODE+'_fill_loop_opm.pdb'
! cp /Users/evabertalan/Downloads/*_fill_loop.pdb $new_loop_opm_name

In [ ]:
! rm /Users/evabertalan/Downloads/*_fill.pdb
! rm /Users/evabertalan/Downloads/*_fill_loop.pdb

### 9. compare the oriented structure with the original opm:

In [22]:
original_opm = '../opm/'+PDB_CODE+'.pdb'

In [25]:
! /Applications/Chimera.app/Contents/MacOS/chimera $directory/$new_loop_opm_name $original_opm

### 10. remove HETATMs from pdb:

In [26]:
with open(new_loop_opm_name) as opm_file:
    input_file = open(directory+'/'+PDB_CODE+'_inp.pdb', 'w+')
    for i, line in enumerate(opm_file):
        if not re.match('HETATM', line):
            input_file.write(line)
    input_file.write('END')
    input_file.close()

In [87]:
inp_file = PDB_CODE+'_inp.pdb'
! open -e $directory/$inp_file

In [118]:
view = nv.show_file(directory+'/'+PDB_CODE+'_inp.pdb')
view

A Jupyter Widget

### 11. upload to charmm-gui:
http://www.charmm-gui.org/?doc=input/membrane

In [50]:
charmm_folder = '/Users/evabertalan/Downloads/charmm-gui.tgz'

#### after step 3 check packing:

In [83]:
tar = tarfile.open(charmm_folder, 'r:gz')
for member in tar.getmembers():
    if re.search('step3_packing.pdb', member.name):
        f = tar.extract(member, 'temp')
f = [i for i in os.listdir('temp') if re.match('charmm-gui', i)]
step3_pdb = 'temp/'+f[0]+'/step3_packing.pdb'

In [148]:
ref = mda.Universe(original_opm).atoms[1:281]
# sel1 = ref.select_atoms('resid 1 281')
mobile = mda.Universe(step3_pdb).atoms[0:280]
# sel2 = mobile.select_atoms('resid 0 280')
aligned = align.alignto(ref, mobile,  weights='mass', tol_mass=100)

In [149]:
view = nv.NGLWidget()
view.add_structure(nv.FileStructure(step3_pdb))
view.add_structure(aligned)
view.update_cartoon(opacity=0.5, component=0, color='blue')
# view = nv.show_file(step3_pdb)
# view = nv.show_file(original_opm)
view

ValueError: (14.030917330729306, 10.016871656718125) is not an instance of Structure

### 12. prepare charmm-gui to NAMD 